In [10]:
import io
import os
from numpy import random
from google.cloud import vision
from Pilow_Utility import draw_borders, Image
import cv2
from PIL import ImageFilter
import pandas as pd

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'starlit-vim-358305-4ba32595705a.json'
client = vision.ImageAnnotatorClient()

file_name = "wakeupcat.jpg"
image_path = os.path.join(file_name)


with io.open(image_path, 'rb') as image_file:
    content = image_file.read()

image = vision.Image(content=content)
response = client.object_localization(image=image).localized_object_annotations

crop_hints_params = vision.CropHintsParams(aspect_ratios=[1.77])
image_context = vision.ImageContext(crop_hints_params=crop_hints_params)


# 이미지 자르기
response2 = client.crop_hints(image=image, image_context=image_context)
hints = response2.crop_hints_annotation.crop_hints

for n, hint in enumerate(hints):
    print('\nCrop Hint: {}'.format(n))

    vertices = (['({},{})'.format(vertex.x, vertex.y)
                for vertex in hint.bounding_poly.vertices])

    print('bounds: {}'.format(','.join(vertices)))

if response2.error.message:
    raise Exception('{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(response.error.message))


# 이미지에 탐지된 객체 박스 생성 (색 랜덤)
pillow_image = Image.open(image_path)
for obj in response:
    r, g, b = random.randint(150, 255), random.randint(150, 255), random.randint(150, 255)

    # draw_borders(pillow_image, obj.bounding_poly, (r, g, b), pillow_image.size, obj.name, obj.score)
    if(obj.name == "Car" or obj.name == "Person" or obj.name == "License plate"):
        draw_borders(pillow_image, obj.bounding_poly, (r, g, b), pillow_image.size, obj.name, obj.score)


# 이미지 객체 박스 좌표를 터미널에서 알려주는 기능
for object_ in response:
    print('\n{} (confidence: {})'.format(object_.name, object_.score))
    print('Normalized bounding polygon vertices: ')
    for vertex in object_.bounding_poly.normalized_vertices:
        print(' - ({}, {})'.format(vertex.x, vertex.y))




# # Names of likelihood from google.cloud.vision.enums
# response3 = client.face_detection(image=image)
# faces = response3.face_annotations
#
# likelihood_name = ('UNKNOWN', 'VERY_UNLIKELY', 'UNLIKELY', 'POSSIBLE',
#                        'LIKELY', 'VERY_LIKELY')
# print('Faces:')
#
# for face in faces:
#     print('anger: {}'.format(likelihood_name[face.anger_likelihood]))
#     print('joy: {}'.format(likelihood_name[face.joy_likelihood]))
#     print('surprise: {}'.format(likelihood_name[face.surprise_likelihood]))
#
#     vertices = (['({},{})'.format(vertex.x, vertex.y) for vertex in face.bounding_poly.vertices])
#
#     print('face bounds: {}'.format(','.join(vertices)))
#
# if response3.error.message:
#     raise Exception('{}\nFor more info on error messages, check: '
#             'https://cloud.google.com/apis/design/errors'.format(response.error.message))
#

pillow_image.show()

ModuleNotFoundError: No module named 'Pilow_Utility'

In [ ]:
from PIL import Image, ImageDraw, Image, ImageFont, ImageFilter

def draw_borders(pillow_image, bounding, color, image_size, caption='', confidence_score=0):
    width, height = image_size
    draw = ImageDraw.Draw(pillow_image)
    draw.polygon([
        bounding.normalized_vertices[0].x * width,
        bounding.normalized_vertices[0].y * height,
        bounding.normalized_vertices[1].x * width,
        bounding.normalized_vertices[1].y * height,
        bounding.normalized_vertices[2].x * width,
        bounding.normalized_vertices[2].y * height,
        bounding.normalized_vertices[3].x * width,
        bounding.normalized_vertices[3].y * height], fill=None, outline=color)

    font_size = width * height // 22000 if width * height > 400000 else 12

    font = ImageFont.truetype(r'C:\Users\CJM\AppData\Local\Microsoft\Windows\Fonts\KoPube돋움.ttf', 12)

    draw.text((bounding.normalized_vertices[0].x * width,
                bounding.normalized_vertices[0].y * height), font=font, text=caption, fill=color)

    # draw.text((bounding.normalized_vertices[0].x * width, bounding.normalized_vertices[0].y * height + 20),
    #           font=font, text='Confidence score: {0:.2f}%'
    #                           ''.format(confidence_score), fill=color)

    return pillow_image